### BayFlux Demo Notebook: Create model and data files representing Fig. 3 of Antoniewicz 2007
By Tyler W. H. Backman

This notebook represents the creation of model and data files for representing the simple test model in Fig. 3 of Antoniewicz 2007. We intend this notebook to serve as a template for use in more complicated projects. For subsequent sampling with BayFlux (in a separate accompanying notebook), the following four files are created here:
* **File 1:** Metabolic model (.sbml format)
* **File 2:** Extracellular exchange and flux bounds (.csv format)
* **File 3:** Atom transitions for each reaction (.txt format)
* **File 4:** Mass distribution experimental data (.tab format)

Citation:

Antoniewicz MR, Kelleher JK, Stephanopoulos G. Elementary metabolite units (EMU): a novel framework for modeling isotopic distributions. Metab Eng. 2007;9(1):68-86. doi:10.1016/j.ymben.2006.09.001

#### Load Python libraries

In [1]:
import bayflux
import cobra
import os

#### Filenames for data export

In [2]:
# Data directory
dataDir = 'input_data'

# File 1: Metabolic model (.sbml format)
modelFile = os.path.join(dataDir, 'fig3toyModel.xml')

# File 2: Extracellular exchange and flux bounds (.csv format)
fluxBoundsFile = os.path.join(dataDir, 'fig3toyFluxBounds.csv')

# File 3: Atom transitions for each reaction (.txt format)
transitionsFile = os.path.join(dataDir, 'fig3toyTransitions.txt')

# File 4: Mass distribution experimental data (.tab format)
mdvFile = os.path.join(dataDir, 'fig3toyMassDistribution.tab')

#### Create File 1: Metabolic model (.sbml format)

We will use cobrapy to build a model corresponding to the reaction stoichiometry in Fig. 3 of Antoniewicz 2007 which consists of 6 metabolites named A-F, and 5 reactions. The figure below is from the manuscript, and shows the structure of this model. Note that for BayFlux, we must produce a valid "genome scale" model, so it is necessary to add extracellular exchange fluxes for each metabolite that exchanges carbon with the outside of the model.

<img src="input_data/fig3.png" alt="" width="250"/>

In [3]:
# create a blank model
cobrapymodel = cobra.Model('fig3')

# define metabolites
A = cobra.Metabolite(
    'A',
    formula='C3',
    name='A',
    compartment='c')
B = cobra.Metabolite(
    'B',
    formula='C3',
    name='B',
    compartment='c')
C = cobra.Metabolite(
    'C',
    formula='C2',
    name='C',
    compartment='c')
D = cobra.Metabolite(
    'D',
    formula='C3',
    name='D',
    compartment='c')
E = cobra.Metabolite(
    'E',
    formula='C1',
    name='E',
    compartment='c')
F = cobra.Metabolite(
    'F',
    formula='C3',
    name='F',
    compartment='c')

# define reactions and add them to the model
a_b = cobra.Reaction('a_b')
a_b.lower_bound = 0
a_b.upper_bound = 500
a_b.add_metabolites({
    A: -1.0,
    B: 1.0
})
print(a_b.reaction)

b_ec = cobra.Reaction('b_ec')
b_ec.lower_bound = 0
b_ec.upper_bound = 500
b_ec.add_metabolites({
    B: -1.0,
    E: 1.0,
    C: 1.0
})
print(b_ec.reaction)

bc_de = cobra.Reaction('bc_de')
bc_de.lower_bound = 0
bc_de.upper_bound = 500
bc_de.add_metabolites({
    B: -1.0,
    C: -1.0,
    D: 1.0,
    E: 2.0
})
print(bc_de.reaction)

d_f = cobra.Reaction('d_f')
d_f.lower_bound = 0
d_f.upper_bound = 500
d_f.add_metabolites({
    D: -1.0,
    F: 1.0
})
print(d_f.reaction)

b_d = cobra.Reaction('b_d')
b_d.lower_bound = -500
b_d.upper_bound = 500
b_d.add_metabolites({
    B: -1.0,
    D: 1.0
})
print(b_d.reaction)

cobrapymodel.add_reactions([a_b, b_ec, bc_de, d_f, b_d])

# add exchange reactions
a_exchange = cobra.Reaction('a_exchange')
a_exchange.lower_bound = -500
a_exchange.upper_bound = 0
a_exchange.add_metabolites({
    A: -1.0
})

e_exchange = cobra.Reaction('e_exchange')
e_exchange.lower_bound = 0
e_exchange.upper_bound = 500
e_exchange.add_metabolites({
    E: -1.0
})

f_exchange = cobra.Reaction('f_exchange')
f_exchange.lower_bound = 0
f_exchange.upper_bound = 500
f_exchange.add_metabolites({
    F: -1.0
})
cobrapymodel.add_reactions([a_exchange, e_exchange, f_exchange])

# set an objective 
cobrapymodel.objective = 'd_f'

A --> B
B --> C + E
B + C --> D + 2.0 E
D --> F
B <=> D


Set an optimization objective and confirm that the model can carry flux (e.g. is a valid model) by optimizing with cobrapy

In [4]:
# Set objective 
cobrapymodel.objective = 'd_f'

# Run FBA (Flux Balance Analysis)
cobrapymodel.optimize()

,fluxes,reduced_costs
a_b,500.0,0.0
b_ec,0.0,0.0
bc_de,0.0,0.0
d_f,500.0,0.0
b_d,500.0,0.0
a_exchange,-500.0,0.0
e_exchange,0.0,-0.0
f_exchange,500.0,2.0


Save the model to a file

In [5]:
cobra.io.write_sbml_model(cobrapymodel, modelFile)

#### Create File 2: Extracellular exchange and flux bounds (.csv format)

First, we need to convert the model to a BayFlux ReactionNetwork to enable us to write the exchanges in a BayFlux file format. To set a constraint, we need to 
get the reaction from the new ReactionNetwork again, as it was copied during
the conversion, and converted to a new object type.

In [6]:
model = bayflux.ReactionNetwork(cobrapymodel)

Get the carbon uptake reaction

In [7]:
carbonInputReaction = model.reactions.get_by_id('a_exchange')
carbonInputReaction

Reaction identifier,a_exchange
Name,
Memory address,0x04074b49d10
Stoichiometry,A <-- A <--
GPR,
Lower bound,-500
Upper bound,0


Set bounds to match those in Fig. 3 of Antoniewicz 2007

In [8]:
carbonInputReaction.lower_bound = -100
carbonInputReaction.upper_bound = -100

Export flux bounds to file. By using exchangesOnly=True, we will write the upper and lower bounds for all exchange reactions to the file

In [9]:
model.writeFluxConstraints(fluxBoundsFile, exchangesOnly=True)

#### Create File 3: Atom transitions for each reaction (.txt format)

This .txt format is based on the file format for jQMM, but includes one change: symmetric metabolites are specified by duplicating the reactions. Note that this simple test model doesn't include any reactions regarded as symmetric.

Here we set the atom transitions for each reaction, and show an example of viewing an EnhancedReaction which reports atom transitions

In [10]:
# create dict of metabolites by name
# we use this instead of directly using the metabolite IDs above, because the conversion to
# a bayflux.ReactionNetwork created new metabolite objects
m = {m.id:m for m in model.metabolites}

model.reactions.a_b.transitions = [bayflux.AtomTransition(
        ((m['A'], [1,2,3]),), # reactant labels
        ((m['B'], [1,2,3]),) # product labels
    )]
model.reactions.b_ec.transitions = [bayflux.AtomTransition(
        ((m['B'], [1,2,3]),), # reactant labels
        ((m['E'], [1]), (m['C'], [2,3]),) # product labels
     )]
model.reactions.bc_de.transitions = [bayflux.AtomTransition(
        ((m['B'], [1,2,3]), (m['C'], [4,5]),), # reactant labels
        ((m['E'], [1]), (m['D'], [2,3,4]), (m['E'], [5]),) # product labels
     )]
model.reactions.d_f.transitions = [bayflux.AtomTransition(
        ((m['D'], [1,2,3]),), # reactant labels
        ((m['F'], [1,2,3]),) # product labels
     )]
model.reactions.b_d.transitions = [bayflux.AtomTransition(
        ((m['B'], [1,2,3]),), # reactant labels
        ((m['D'], [1,2,3]),) # product labels
     )]

model.reactions.b_d

Reaction identifier,b_d
Name,
Memory address,0x04074b49c90
Stoichiometry,B <=> D B <=> D
GPR,
Lower bound,-500
Upper bound,500
Atom transition,B --> D abc : abc


Export atom transitions to file

In [11]:
model.writeAtomTransitions(transitionsFile)

#### Create File 4: Mass distribution experimental data (.tab format)

BayFlux stores experimental mass distribution data in a MassDistribution object
which has built in I/O capabilities. The data must first be in the format shown below, as a dictionary where keys are EMU objects, and values are a list of (mass distribution) lists. Multiple mass distributions can be used for the same EMU, which allows for storing independent experimental measurements.

Here we create this directly with the labeling data from Antoniewicz 2007.

In [12]:
m = {m.id:m for m in model.metabolites}

mdvData = {
    bayflux.EMU(m['F'],[0,1,2]): [[0.0001, 0.8008, 0.1983, 0.0009],],
    }

Create and view the MassDistribution object

In [13]:
mdvObject = bayflux.MassDistribution(model, mdvData)
mdvObject

  metabolite  atoms       0       1       2       3
0          F  0,1,2  0.0001  0.8008  0.1983  0.0009

Save to file

In [14]:
mdvObject.writeToFile(mdvFile)

In [15]:
model.exchanges

[<EnhancedReaction a_exchange at 0x4074b49d10>,
 <EnhancedReaction e_exchange at 0x4074b49d50>,
 <EnhancedReaction f_exchange at 0x4074b49d90>]